In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved.

# Ajuster un simple champ de radiance neuronale via raymarching


Ce didacticiel montre comment ajuster le champ de radiance neuronale à un ensemble de vues d'une scène à l'aide du rendu de fonction implicite différentiable.


Plus précisément, ce tutoriel expliquera comment :
1. Créez un moteur de rendu de fonction implicite différenciable avec un échantillonnage de grille d'image ou de rayons de Monte Carlo.
2. Créez un modèle implicite d'une scène.
3. Ajustez la fonction implicite (Neural Radiance Field) en fonction des images d'entrée à l'aide du moteur de rendu implicite différentiable.
4. Visualisez la fonction implicite apprise.


Notez que le modèle implicite présenté est une version simplifiée de NeRF :<br>
_Ben Mildenhall, Pratul P. Srinivasan, Matthew Tancik, Jonathan T. Barron, Ravi Ramamoorthi, Ren Ng : NeRF : Représenter des scènes sous forme de champs de rayonnement neuronal pour la synthèse de vues, ECCV 2020._


Les simplifications incluent :
* *Échantillonnage de rayons* : Ce carnet n'effectue pas d'échantillonnage de rayons stratifié mais plutôt un échantillonnage de rayons à des profondeurs équidistantes.
* *Rendu* : nous effectuons une seule passe de rendu, contrairement à l'implémentation originale qui effectue une passe de rendu grossière et fine.
* *Architecture* : Notre réseau est moins profond, ce qui permet une optimisation plus rapide, éventuellement au détriment des détails de surface.
* *Perte de masque* : Puisque nos observations incluent des masques de segmentation, nous optimisons également une perte de silhouette qui oblige les rayons soit à être entièrement absorbés à l'intérieur du volume, soit à le traverser complètement.

## 0. Install and Import modules
Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [2]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("2.1.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=5ca404a633fceb3e2e62cb4ff463f7b933b6d3a124718f6ffa86df6eb8edccca
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=e1262871c300db8baff8c61f64b82ae27b9f9057411b7490b33b7204eb55bb33
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt210/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 165.3 MB/s eta 0:00:00


In [3]:
# %matplotlib inline
# %matplotlib notebook
import os
import sys
import time
import json
import glob
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython import display
from tqdm.notebook import tqdm

# Data structures and functions for rendering
from pytorch3d.structures import Volumes
from pytorch3d.transforms import so3_exp_map
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    NDCMultinomialRaysampler,
    MonteCarloRaysampler,
    EmissionAbsorptionRaymarcher,
    ImplicitRenderer,
    RayBundle,
    ray_bundle_to_ray_points,
)

# obtain the utilized device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    print(
        'Please note that NeRF is a resource-demanding method.'
        + ' Running this notebook on CPU will be extremely slow.'
        + ' We recommend running the example on a GPU'
        + ' with at least 10 GB of memory.'
    )
    device = torch.device("cpu")

In [4]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/generate_cow_renders.py
from plot_image_grid import image_grid
from generate_cow_renders import generate_cow_renders

--2023-12-05 12:47:49--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.57K  --.-KB/s    in 0s      

2023-12-05 12:47:49 (33.2 MB/s) - ‘plot_image_grid.py’ saved [1608/1608]

--2023-12-05 12:47:49--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/generate_cow_renders.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request se

OU en cas d'exécution locale, décommentez et exécutez la cellule suivante :

In [5]:
# from utils.generate_cow_renders import generate_cow_renders
# from utils import image_grid

## 1. Générer des images de la scène et des masques

La cellule suivante génère nos données d'entraînement.
Il restitue le maillage de vache du didacticiel `fit_textured_mesh.ipynb` sous plusieurs points de vue et renvoie :
1. Un lot de tenseurs d'image et de silhouette produits par le moteur de rendu de maillage de vache.
2. Un ensemble de caméras correspondant à chaque rendu.

Remarque : Pour les besoins de ce tutoriel, qui vise à expliquer les détails du rendu implicite, nous n'expliquons pas comment fonctionne le rendu de maillage, implémenté dans la fonction `generate_cow_renders`. Veuillez vous référer à `fit_textured_mesh.ipynb` pour une explication détaillée du rendu du maillage.

In [6]:
target_cameras, target_images, target_silhouettes = generate_cow_renders(num_views=40, azimuth_range=180)
print(f'Generated {len(target_images)} images/silhouettes/cameras.')

Generated 40 images/silhouettes/cameras.


## 2. Initialisez le moteur de rendu implicite

Ce qui suit initialise un moteur de rendu implicite qui émet un rayon à partir de chaque pixel d'une image cible et échantillonne un ensemble de points uniformément espacés le long du rayon. À chaque point de rayon, la densité et la valeur de couleur correspondantes sont obtenues en interrogeant l'emplacement correspondant dans le modèle neuronal de la scène (le modèle est décrit et instancié dans une cellule ultérieure).

Le moteur de rendu est composé d'un *raymarcher* et d'un *raysampler*.
- Le *raysampler* est chargé d'émettre des rayons à partir des pixels de l'image et d'échantillonner les points le long d'eux. Ici, nous utilisons deux rayamplers différents :
     - `MonteCarloRaysampler` est utilisé pour générer des rayons à partir d'un sous-ensemble aléatoire de pixels du plan image. Le sous-échantillonnage aléatoire des pixels est effectué lors de l'**entraînement** pour diminuer la consommation mémoire du modèle implicite.
     - `NDCMultinomialRaysampler` qui suit la convention standard de grille de coordonnées PyTorch3D (+X de droite à gauche ; +Y de bas en haut ; +Z loin de l'utilisateur). En combinaison avec le modèle implicite de la scène, `NDCMultinomialRaysampler` consomme une grande quantité de mémoire et, par conséquent, n'est utilisé que pour visualiser les résultats de l'entraînement au moment du **test**.
- Le *raymarcher* prend les densités et les couleurs échantillonnées le long de chaque rayon et restitue chaque rayon en une couleur et une valeur d'opacité du pixel source du rayon. Ici, nous utilisons le `EmissionAbsorptionRaymarcher` qui implémente l'algorithme standard de raymarching Emission-Absorption.

In [7]:
# render_size décrit la taille des deux côtés du
# images rendues en pixels. Puisqu'un avantage de
# Les champs de rayonnement neuronal sont des rendus de haute qualité
# avec une quantité importante de détails, nous rendons
# la fonction implicite au double de la taille de
# images cibles.
render_size = target_images.shape[1] * 2

# Notre scène rendue est centrée autour de (0,0,0)
# et est enfermé dans un cadre de délimitation
# dont le côté est à peu près égal à 3,0 (unités mondiales).
volume_extent_world = 3.0

# 1) Instanciez les rayamplers.

# Ici, NDCMultinomialRaysampler génère une image rectangulaire
# grille de rayons dont les coordonnées suivent le PyTorch3D
# conventions de coordonnées.
raysampler_grid = NDCMultinomialRaysampler(
    image_height=render_size,
    image_width=render_size,
    n_pts_per_ray=128,
    min_depth=0.1,
    max_depth=volume_extent_world,
)

# MonteCarloRaysampler génère un sous-ensemble aléatoire
# de rayons `n_rays_per_image` émis depuis le plan image.
raysampler_mc = MonteCarloRaysampler(
    min_x = -1.0,
    max_x = 1.0,
    min_y = -1.0,
    max_y = 1.0,
    n_rays_per_image=750,
    n_pts_per_ray=128,
    min_depth=0.1,
    max_depth=volume_extent_world,
)

# 2) Instanciez le raymarcher.
# Ici, nous utilisons le standard EmissionAbsorptionRaymarcher
# qui marche le long de chaque rayon afin de rendre
# le rayon en un seul vecteur de couleur 3D
# et un scalaire d'opacité.
raymarcher = EmissionAbsorptionRaymarcher()

# Enfin, instanciez les rendus implicites
# pour les deux rayamplers.
renderer_grid = ImplicitRenderer(
    raysampler=raysampler_grid, raymarcher=raymarcher,
)
renderer_mc = ImplicitRenderer(
    raysampler=raysampler_mc, raymarcher=raymarcher,
)

## 3. Définir le modèle de champ de radiance neuronale

Dans cette cellule, nous définissons le module `NeuralRadianceField`, qui spécifie un champ continu de couleurs et d'opacités sur le domaine 3D de la scène.

La fonction `forward` de `NeuralRadianceField` (NeRF) reçoit en entrée un ensemble de tenseurs qui paramétrent un faisceau de rayons de rendu. Le faisceau de rayons est ensuite converti en points de rayons 3D dans les coordonnées mondiales de la scène. Chaque point 3D est ensuite mappé à une représentation harmonique à l'aide du calque `HarmonicEmbedding` (défini dans la cellule suivante). Les intégrations harmoniques entrent ensuite dans les branches _color_ et _opacity_ du modèle NeRF afin d'étiqueter chaque point de rayon avec un vecteur 3D et un scalaire 1D compris entre [0-1] qui définissent respectivement la couleur RVB et l'opacité du point.

Étant donné que NeRF a une grande empreinte mémoire, nous implémentons également la méthode « NeuralRadianceField.forward_batched ». La méthode divise les rayons d'entrée en lots et exécute la fonction « forward » pour chaque lot séparément dans une boucle for. Cela nous permet de restituer un grand nombre de rayons sans manquer de mémoire GPU. Standardement, `forward_batched` serait utilisé pour restituer les rayons émis par tous les pixels d'une image afin de produire un rendu en taille réelle d'une scène.

In [8]:
class HarmonicEmbedding(torch.nn.Module):
    def __init__(self, n_harmonic_functions=60, omega0=0.1):
        """
         Étant donné un tenseur d'entrée `x` de forme [minibatch, ... , dim],
         la couche d'intégration harmonique convertit chaque fonctionnalité
         dans `x` dans une série de caractéristiques harmoniques `embedding`
         comme suit:
            embedding[..., i*dim:(i+1)*dim] = [
                sin(x[..., i]),
                sin(2*x[..., i]),
                sin(4*x[..., i]),
                ...
                sin(2**(self.n_harmonic_functions-1) * x[..., i]),
                cos(x[..., i]),
                cos(2*x[..., i]),
                cos(4*x[..., i]),
                ...
                cos(2**(self.n_harmonic_functions-1) * x[..., i])
            ]

        Notez que `x` est également prémultiplié par `omega0` avant
         évaluer les fonctions harmoniques.
        """
        super().__init__()
        self.register_buffer(
            'frequencies',
            omega0 * (2.0 ** torch.arange(n_harmonic_functions)),
        )
    def forward(self, x):
        """
        Args:
            x: tensor of shape [..., dim]
        Returns:
            embedding: a harmonic embedding of `x`
                of shape [..., n_harmonic_functions * dim * 2]
        """
        embed = (x[..., None] * self.frequencies).view(*x.shape[:-1], -1)
        return torch.cat((embed.sin(), embed.cos()), dim=-1)


class NeuralRadianceField(torch.nn.Module):
    def __init__(self, n_harmonic_functions=60, n_hidden_neurons=256):
        super().__init__()
        """
         Args :
             n_harmonic_functions : Le nombre de fonctions harmoniques
                 utilisé pour former l’intégration harmonique de chaque point.
             n_hidden_neurons : le nombre d'unités cachées dans le
                 couches entièrement connectées des MLP du modèle.
         """

         # La couche d'intégration harmonique convertit les coordonnées 3D d'entrée
         # à une représentation plus adaptée à
         # traitement avec un réseau neuronal profond.
        self.harmonic_embedding = HarmonicEmbedding(n_harmonic_functions)

        # La dimension de l'intégration harmonique.
        embedding_dim = n_harmonic_functions * 2 * 3

        # self.mlp est un simple perceptron multicouche à 2 couches
         # qui convertit les intégrations harmoniques d'entrée par point
         # à une représentation latente.
         # Non pas que nous utilisions les activations Softplus au lieu de ReLU.
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(embedding_dim, n_hidden_neurons),
            torch.nn.Softplus(beta=10.0),
            torch.nn.Linear(n_hidden_neurons, n_hidden_neurons),
            torch.nn.Softplus(beta=10.0),
        )

        # Fonctionnalités données prédites par self.mlp, self.color_layer
         # est chargé de prédire un vecteur 3D par point
         # qui représente la couleur RVB du point.
        self.color_layer = torch.nn.Sequential(
            torch.nn.Linear(n_hidden_neurons + embedding_dim, n_hidden_neurons),
            torch.nn.Softplus(beta=10.0),
            torch.nn.Linear(n_hidden_neurons, 3),
            torch.nn.Sigmoid(),
            # Pour garantir que les couleurs se situent correctement entre [0-1],
             # la couche se termine par une couche sigmoïde.
        )

        # La couche de densité convertit les caractéristiques de self.mlp
         # à une valeur de densité 1D représentant l'opacité brute
         # de chaque point.
        self.density_layer = torch.nn.Sequential(
            torch.nn.Linear(n_hidden_neurons, 1),
            torch.nn.Softplus(beta=10.0),
            # L'activation de Sofplus garantit que l'opacité brute
             # est un nombre non négatif.
        )

        # Nous fixons le biais de la couche de densité à -1,5
         # afin d'initialiser les opacités des
         # ray pointe vers des valeurs proches de 0.
         # C'est un détail crucial pour assurer la convergence
         # du modèle.
        self.density_layer[0].bias.data[0] = -1.5

    def _get_densities(self, features):
        """
        Cette fonction prend les « fonctionnalités » prédites par « self.mlp »
         et les convertit en `raw_densities` avec `self.density_layer`.
         Les « raw_densities » sont ensuite mappées sur la plage [0-1] avec
         1 - exponentielle inverse de `raw_densities`.
        """
        raw_densities = self.density_layer(features)
        return 1 - (-raw_densities).exp()

    def _get_colors(self, features, rays_directions):
        """
        Cette fonction prend les « fonctionnalités » par point prédites par « self.mlp »
         et évalue le modèle de couleur afin de l'attacher à chacun
         pointez un vecteur 3D de sa couleur RVB.

         Afin de représenter les effets dépendants du point de vue,
         avant d'évaluer `self.color_layer`, `NeuralRadianceField`
         concatène aux « caractéristiques » un intégration harmonique
         de `ray_directions`, qui sont des directions par point
         de rayons ponctuels exprimés sous forme de vecteurs 3D normalisés l2
         en coordonnées mondiales.
        """
        spatial_size = features.shape[:-1]

        # Normalisez les ray_directions à la norme de l'unité l2.
        rays_directions_normed = torch.nn.functional.normalize(
            rays_directions, dim=-1
        )

        # Obtenez l'intégration harmonique des directions des rayons normalisées.
        rays_embedding = self.harmonic_embedding(
            rays_directions_normed
        )

        # Développez le tenseur des directions des rayons afin que sa taille spatiale
         # est égal à la taille des fonctionnalités.
        rays_embedding_expand = rays_embedding[..., None, :].expand(
            *spatial_size, rays_embedding.shape[-1]
        )

        # Concaténer les intégrations de direction de rayon avec
         # fonctionnalités et évaluer le modèle de couleur.
        color_layer_input = torch.cat(
            (features, rays_embedding_expand),
            dim=-1
        )
        return self.color_layer(color_layer_input)


    def forward(
        self,
        ray_bundle: RayBundle,
        **kwargs,
    ):
        """
        La fonction forward accepte les paramétrages de
         Points 3D échantillonnés le long des rayons de projection. L'avant
         pass est responsable de l'attachement d'un vecteur 3D
         et un scalaire 1D représentant le point
         Couleur RVB et opacité respectivement.

         Args :
             ray_bundle : un objet RayBundle contenant les variables suivantes :
                 origines : Un tenseur de forme `(minibatch, ..., 3)` désignant le
                     origines des rayons d'échantillonnage dans les coordonnées mondiales.
                 directions : Un tenseur de forme `(minibatch, ..., 3)`
                     contenant les vecteurs de direction des rayons d'échantillonnage en coordonnées mondiales.
                 lengths : Un tenseur de forme `(minibatch, ..., num_points_per_ray)`
                     contenant les longueurs auxquelles les rayons sont échantillonnés.

        Returns:
            rays_densities: A tensor of shape `(minibatch, ..., num_points_per_ray, 1)`
                denoting the opacity of each ray point.
            rays_colors: A tensor of shape `(minibatch, ..., num_points_per_ray, 3)`
                denoting the color of each ray point.
        """
        # Nous convertissons d'abord les paramètres des rayons en coordonnées mondiales avec `ray_bundle_to_ray_points`.
        # coordonnées mondiales avec `ray_bundle_to_ray_points`.
        rays_points_world = ray_bundle_to_ray_points(ray_bundle)
        # rays_points_world.shape = [minibatch x ... x 3]

        # Pour chaque coordonnée 3D du monde, nous obtenons son intégration harmonique.
        embeds = self.harmonic_embedding(
            rays_points_world
        )
        # embeds.shape = [minibatch x ... x self.n_harmonic_functions*6]

        # self.mlp fait correspondre chaque intégration harmonique à un espace de caractéristiques latentes.
        features = self.mlp(embeds)
        # features.shape = [minibatch x ... x n_hidden_neurons]

        # Enfin, étant donné les caractéristiques par point,
        # exécuter les branches densité et couleur.

        rays_densities = self._get_densities(features)
        # rays_densities.shape = [minibatch x ... x 1]

        rays_colors = self._get_colors(features, ray_bundle.directions)
        # rays_colors.shape = [minibatch x ... x 3]

        return rays_densities, rays_colors

    def batched_forward(
        self,
        ray_bundle: RayBundle,
        n_batches: int = 16,
        **kwargs,
    ):
        """
        Cette fonction est utilisée pour permettre un traitement efficace de la mémoire
        des rayons d'entrée. Les rayons d'entrée sont d'abord divisés en morceaux `n_batches`
        et sont passés à travers la fonction `self.forward` un par un
        dans une boucle for. Combiné avec la désactivation de la mise en cache du gradient de PyTorch
        (`torch.no_grad()`), cela permet de rendre de grands lots de
        de rayons qui ne tiennent pas tous dans la mémoire du GPU en une seule passe.
        Dans notre cas, batched_forward est utilisé pour exporter un rendu complet du champ de radiance pour la visualisation.
        du champ de radiance à des fins de visualisation.

        Args:
            ray_bundle : Un objet RayBundle contenant les variables suivantes :
                origines : Un tenseur de la forme `(minibatch, ..., 3)` représentant les
                    origines des rayons d'échantillonnage en coordonnées mondiales.
                directions : Un tenseur de forme `(minibatch, ..., 3)`
                    contenant les vecteurs de direction des rayons d'échantillonnage en coordonnées mondiales.
                lengths : Un tenseur de forme `(minibatch, ..., num_points_per_ray)`
                    contenant les longueurs auxquelles les rayons sont échantillonnés.
            n_batches : Spécifie le nombre de lots dans lesquels les rayons d'entrée sont divisés.
                Plus le nombre de lots est élevé, plus l'empreinte mémoire est faible et plus la vitesse de traitement est réduite.
                et plus la vitesse de traitement est faible.

        Returns:
            rays_densities : Un tenseur de la forme `(minibatch, ..., num_points_per_ray, 1)`
                indiquant l'opacité de chaque point du rayon.
            rays_colors : Un tenseur de forme `(minibatch, ..., num_points_par_ray, 3)`
                indiquant la couleur de chaque point de rayon.

        """

        #Cette fonction analyse les formes nécessaires au remodelage du tenseur.
        n_pts_per_ray = ray_bundle.lengths.shape[-1]
        spatial_size = [*ray_bundle.origins.shape[:-1], n_pts_per_ray]

        # Divisez les rayons en lots `n_batches`.
        tot_samples = ray_bundle.origins.shape[:-1].numel()
        batches = torch.chunk(torch.arange(tot_samples), n_batches)

        # Pour chaque lot, exécutez la passe avant standard.
        batch_outputs = [
            self.forward(
                RayBundle(
                    origins=ray_bundle.origins.view(-1, 3)[batch_idx],
                    directions=ray_bundle.directions.view(-1, 3)[batch_idx],
                    lengths=ray_bundle.lengths.view(-1, n_pts_per_ray)[batch_idx],
                    xys=None,
                )
            ) for batch_idx in batches
        ]

        # Concaténer les rayons_densités et les rayons_colors par lot
         # et remodeler en fonction des tailles des entrées.
        rays_densities, rays_colors = [
            torch.cat(
                [batch_output[output_i] for batch_output in batch_outputs], dim=0
            ).view(*spatial_size, -1) for output_i in (0, 1)
        ]
        return rays_densities, rays_colors

## 4. Fonctions d'assistance

Dans cette fonction, nous définissons des fonctions qui aident à l'optimisation du champ de radiance neuronale.

In [9]:
def huber(x, y, scaling=0.1):
    """
     Une fonction d'assistance pour évaluer la perte douce de L1 (huber)
     entre les silhouettes et les couleurs rendues.
     """
    diff_sq = (x - y) ** 2
    loss = ((1 + diff_sq / (scaling**2)).clamp(1e-4).sqrt() - 1) * float(scaling)
    return loss

def sample_images_at_mc_locs(target_images, sampled_rays_xy):
    """
     Étant donné un ensemble d'emplacements de pixels de Monte Carlo `sampled_rays_xy`,
     cette méthode échantillonne le tenseur `target_images` au
     emplacements 2D respectifs.

     Cette fonction est utilisée afin d'extraire les couleurs de
     des images de vérité terrain qui correspondent aux couleurs
     rendu à l'aide de `MonteCarloRaysampler`.
     """
    ba = target_images.shape[0]
    dim = target_images.shape[-1]
    spatial_size = sampled_rays_xy.shape[1:-1]
    # Afin d'échantillonner target_images, nous utilisons
     # la fonction grid_sample qui implémente un
     # échantillonneur d'images bilinéaires.
     # Notez qu'il faut inverser le signe du
     # positions de rayons échantillonnées pour convertir les emplacements NDC xy
     # du MonteCarloRaysampler à la coordonnée
     # convention de grid_sample.
    images_sampled = torch.nn.functional.grid_sample(
        target_images.permute(0, 3, 1, 2),
        -sampled_rays_xy.view(ba, -1, 1, 2),  # note the sign inversion
        align_corners=True
    )
    return images_sampled.permute(0, 2, 3, 1).view(
        ba, *spatial_size, dim
    )

def show_full_render(
    neural_radiance_field, camera,
    target_image, target_silhouette,
    loss_history_color, loss_history_sil,
):
    """
    Il s'agit d'une fonction d'assistance pour visualiser le
     résultats intermédiaires de l’apprentissage.

     Puisque le `NeuralRadianceField` souffre de
     une empreinte mémoire importante, qui ne nous permet pas
     rendre la grille d'image complète en un seul passage,
     nous utilisons le `NeuralRadianceField.batched_forward`
     fonction en combinaison avec la désactivation de la mise en cache du dégradé.
     Cela divise l'ensemble des rayons émis en lots et
     évalue la fonction implicite sur un lot à la fois
     pour éviter le débordement de la mémoire GPU.
    """

    # Empêcher la mise en cache des dégradés.
    with torch.no_grad():
        # Rendu en utilisant le moteur de rendu de grille et le
         # Fonction batched_forward de neural_radiance_field.
        rendered_image_silhouette, _ = renderer_grid(
            cameras=camera,
            volumetric_function=neural_radiance_field.batched_forward
        )
        # Diviser le résultat du rendu en un rendu de silhouette
         # et le rendu de l'image.
        rendered_image, rendered_silhouette = (
            rendered_image_silhouette[0].split([3, 1], dim=-1)
        )

    # Générer des tracés.
    fig, ax = plt.subplots(2, 3, figsize=(15, 10))
    ax = ax.ravel()
    clamp_and_detach = lambda x: x.clamp(0.0, 1.0).cpu().detach().numpy()
    ax[0].plot(list(range(len(loss_history_color))), loss_history_color, linewidth=1)
    ax[1].imshow(clamp_and_detach(rendered_image))
    ax[2].imshow(clamp_and_detach(rendered_silhouette[..., 0]))
    ax[3].plot(list(range(len(loss_history_sil))), loss_history_sil, linewidth=1)
    ax[4].imshow(clamp_and_detach(target_image))
    ax[5].imshow(clamp_and_detach(target_silhouette))
    for ax_, title_ in zip(
        ax,
        (
            "loss color", "rendered image", "rendered silhouette",
            "loss silhouette", "target image",  "target silhouette",
        )
    ):
        if not title_.startswith('loss'):
            ax_.grid("off")
            ax_.axis("off")
        ax_.set_title(title_)
    fig.canvas.draw(); fig.show()
    display.clear_output(wait=True)
    display.display(fig)
    return fig


## 5. Ajuster le champ de radiance

Nous réalisons ici l'ajustement du champ de radiance avec un rendu différentiable.

Afin d'ajuster le champ de radiance, nous le rendons du point de vue des `target_cameras`
et comparez les rendus résultants avec les «target_images» et «target_silhouettes» observées.

La comparaison est effectuée en évaluant l'erreur moyenne de Huber (smooth-l1) entre les
paires de `target_images`/`rendered_images` et `target_silhouettes`/`rendered_silhouettes`.

Puisque nous utilisons le `MonteCarloRaysampler`, les sorties du moteur de rendu de formation `renderer_mc`
sont des couleurs de pixels échantillonnés aléatoirement dans le plan de l'image, et non un réseau de pixels formant
une image valide. Ainsi, afin de comparer les couleurs rendues avec la vérité terrain, nous
utiliser les emplacements aléatoires des pixels de MonteCarlo pour échantillonner les images/silhouettes de vérité terrain
`target_silhouettes`/`rendered_silhouettes` aux emplacements xy correspondant au rendu
Emplacements. Cela se fait avec la fonction d'assistance `sample_images_at_mc_locs`, qui est
décrit dans la cellule précédente.

In [ ]:
# Déplacez d'abord toutes les variables pertinentes vers le bon appareil.
renderer_grid = renderer_grid.to(device)
renderer_mc = renderer_mc.to(device)
target_cameras = target_cameras.to(device)
target_images = target_images.to(device)
target_silhouettes = target_silhouettes.to(device)

# Définir la graine pour la reproductibilité
torch.manual_seed(1)

# Instanciez le modèle de champ de radiance.
neural_radiance_field = NeuralRadianceField().to(device)

# Instanciez l'optimiseur Adam. Nous avons fixé son taux d'apprentissage principal à 1e-3.
lr = 1e-3
optimizer = torch.optim.Adam(neural_radiance_field.parameters(), lr=lr)

# Nous échantillonnons 6 caméras aléatoires dans un mini-lot. Chaque caméra
# émet des rayonsrayampler_mc.n_pts_per_image.
batch_size = 6

# 3000 itérations prennent environ 20 minutes sur une Tesla M40 et conduisent à
# des résultats raisonnablement nets. Cependant, pour le meilleur possible
# résultats, nous vous recommandons de définir n_iter=20000.
n_iter = 3000

# Initialisez les tampons de l'historique des pertes.
loss_history_color, loss_history_sil = [], []

# La boucle d'optimisation principale.
for iteration in range(n_iter):
    # Dans le cas où nous aurions atteint les derniers 75 % d'itérations,
     # diminuer le taux d'apprentissage de l'optimiseur de 10 fois.
    if iteration == round(n_iter * 0.75):
        print('Decreasing LR 10-fold ...')
        optimizer = torch.optim.Adam(
            neural_radiance_field.parameters(), lr=lr * 0.1
        )

    # Zéro le gradient de l'optimiseur.
    optimizer.zero_grad()

    # Échantillon d'indices de lots aléatoires.
    batch_idx = torch.randperm(len(target_cameras))[:batch_size]

    # Échantillonnez le minilot de caméras.
    batch_cameras = FoVPerspectiveCameras(
        R = target_cameras.R[batch_idx],
        T = target_cameras.T[batch_idx],
        znear = target_cameras.znear[batch_idx],
        zfar = target_cameras.zfar[batch_idx],
        aspect_ratio = target_cameras.aspect_ratio[batch_idx],
        fov = target_cameras.fov[batch_idx],
        device = device,
    )

    # Évaluez le modèle nerf.
    rendered_images_silhouettes, sampled_rays = renderer_mc(
        cameras=batch_cameras,
        volumetric_function=neural_radiance_field
    )
    rendered_images, rendered_silhouettes = (
        rendered_images_silhouettes.split([3, 1], dim=-1)
    )

    # Calculez l'erreur de silhouette comme le huber moyen
     # perte entre les masques prédits et le
     # silhouettes cibles échantillonnées.
    silhouettes_at_rays = sample_images_at_mc_locs(
        target_silhouettes[batch_idx, ..., None],
        sampled_rays.xys
    )
    sil_err = huber(
        rendered_silhouettes,
        silhouettes_at_rays,
    ).abs().mean()

    # Calculez l'erreur de couleur comme le huber moyen
     # perte entre les couleurs rendues et le
     # images cibles échantillonnées.
    colors_at_rays = sample_images_at_mc_locs(
        target_images[batch_idx],
        sampled_rays.xys
    )
    color_err = huber(
        rendered_images,
        colors_at_rays,
    ).abs().mean()

    # La perte d'optimisation est simple
     # somme des erreurs de couleur et de silhouette.
    loss = color_err + sil_err

    # Enregistrez l'historique des pertes.
    loss_history_color.append(float(color_err))
    loss_history_sil.append(float(sil_err))

    # Toutes les 10 itérations, imprimez les valeurs actuelles des pertes.
    if iteration % 10 == 0:
        print(
            f'Iteration {iteration:05d}:'
            + f' loss color = {float(color_err):1.2e}'
            + f' loss silhouette = {float(sil_err):1.2e}'
        )

    # Passez à l'étape d'optimisation.
    loss.backward()
    optimizer.step()

    # Visualisez les rendus complets toutes les 100 itérations.
    if iteration % 100 == 0:
        show_idx = torch.randperm(len(target_cameras))[:1]
        show_full_render(
            neural_radiance_field,
            FoVPerspectiveCameras(
                R = target_cameras.R[show_idx],
                T = target_cameras.T[show_idx],
                znear = target_cameras.znear[show_idx],
                zfar = target_cameras.zfar[show_idx],
                aspect_ratio = target_cameras.aspect_ratio[show_idx],
                fov = target_cameras.fov[show_idx],
                device = device,
            ),
            target_images[show_idx][0],
            target_silhouettes[show_idx][0],
            loss_history_color,
            loss_history_sil,
        )

## 6. Visualiser le champ de rayonnement neuronal optimisé

Enfin, nous visualisons le champ de radiance neuronale en effectuant un rendu à partir de plusieurs points de vue qui tournent autour de l'axe y du volume.

In [ ]:
def generate_rotating_nerf(neural_radiance_field, n_frames = 50):
    logRs = torch.zeros(n_frames, 3, device=device)
    logRs[:, 1] = torch.linspace(-3.14, 3.14, n_frames, device=device)
    Rs = so3_exp_map(logRs)
    Ts = torch.zeros(n_frames, 3, device=device)
    Ts[:, 2] = 2.7
    frames = []
    print('Rendering rotating NeRF ...')
    for R, T in zip(tqdm(Rs), Ts):
        camera = FoVPerspectiveCameras(
            R=R[None],
            T=T[None],
            znear=target_cameras.znear[0],
            zfar=target_cameras.zfar[0],
            aspect_ratio=target_cameras.aspect_ratio[0],
            fov=target_cameras.fov[0],
            device=device,
        )
        # Notez que nous rendons à nouveau avec `NDCMultinomialRaysampler`
         # et la fonction batched_forward de neural_radiance_field.
        frames.append(
            renderer_grid(
                cameras=camera,
                volumetric_function=neural_radiance_field.batched_forward,
            )[0][..., :3]
        )
    return torch.cat(frames)

with torch.no_grad():
    rotating_nerf_frames = generate_rotating_nerf(neural_radiance_field, n_frames=3*5)

image_grid(rotating_nerf_frames.clamp(0., 1.).cpu().numpy(), rows=3, cols=5, rgb=True, fill=True)
plt.show()